In [ ]:
import os
import jieba
import mysql.connector
'''
  判断字符串是否为数字，清理包括:1,1.5,023,34%等特别的数字字符串
'''
def isNumber(num):
    if(num.isdigit() or num.isnumeric() or num.isdecimal()):
        return True
    if num.endswith('%'):
        num_str = num[:-1]  # 去掉百分号
        return isNumber(num_str)
    try:
        _ = float(num)
        return True
    except ValueError:
        return False
    return False
'''
  将某个字符串通过jieba分词后通过空格拼接，因为CountVectorizer统计词频传入的是带空格的字符串
'''
def wordToVec(word):
        word_cut = jieba.cut(word, cut_all = False) 
        filtered_words = filter(lambda w: w is not None and len(w.strip()) > 0 and not isNumber(w.strip()), list(word_cut))  # 过滤掉空字符串
        word_list=" ".join(filtered_words)
        return word_list
def LoadData():
    mydb = mysql.connector.connect(
      host="192.168.20.20",
      user="root",
      passwd="db@12345",
      database="gvtgms",
      port="3307"
    )
    mycursor = mydb.cursor()
    mycursor.execute("SELECT CONCAT(p.name,''),c.NAME,h.PTID,h.ClassName FROM gvtgms.product  p LEFT JOIN gvtgms.class c ON p.CLASS_ID=c.ID INNER JOIN apos_splitpackage.productclass h ON h.PTID=p.PT_ID")
    myresult = mycursor.fetchall()
    data_list = []                                                #训练集
    class_list = []
    for x in myresult:
        raw=x[0].decode('utf-8')+x[1]+x[3]
        lable=x[2]
        word_list=wordToVec(raw)
        data_list.append(word_list)
        class_list.append(lable)
    return data_list,class_list

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib
if __name__ == '__main__':
    data_list,class_list=LoadData()
    stop_words="";
    with open(os.path.join("./新闻分类数据", "stopwords_cn.txt"), 'r', encoding = 'utf-8') as f:    #打开txt文件
                stop_words = f.read()
    stop_words_array=stop_words.split("\n") 
    #除了停止词外，单个字母的都会被自动过滤掉
    vectorizer = CountVectorizer(stop_words=stop_words_array)
    X = vectorizer.fit_transform(data_list)
    fn=np.array(vectorizer.get_feature_names())
    print("特征列：",fn)
    v=np.array(X.toarray())
    print("词条向量：\n",v)
    joblib.dump(vectorizer, './vectorizer.pkl')

In [ ]:
    from sklearn.utils import shuffle
    from sklearn.model_selection import train_test_split
    from sklearn.externals import joblib

    X, y = shuffle(v, class_list, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)
    
    classifier = MultinomialNB().fit(X_train, y_train)
    # 将模型保存至文件
    joblib.dump(classifier, './classification_of_goods.pkl')
    print("训练数据模型完成")

In [ ]:
    from sklearn.externals import joblib
    classifier = joblib.load('./classification_of_goods.pkl')
    test_accuracy = classifier.score(X_test, y_test)
    print(test_accuracy)



In [1]:
import jieba
def wordToVec(word):
    word_cut = jieba.cut(word, cut_all = False) 
    filtered_words = filter(lambda w: w is not None and len(w.strip()) > 0 and not isNumber(w.strip()), list(word_cut))  # 过滤掉空字符串
    word_list=" ".join(filtered_words)
    return word_list
'''
  判断字符串是否为数字，清理包括:1,1.5,023,34%等特别的数字字符串
'''
def isNumber(num):
    if(num.isdigit() or num.isnumeric() or num.isdecimal()):
        return True
    if num.endswith('%'):
        num_str = num[:-1]  # 去掉百分号
        return isNumber(num_str)
    try:
        _ = float(num)
        return True
    except ValueError:
        return False
    return False

In [2]:
    from sklearn.externals import joblib
    classifier = joblib.load('./classification_of_goods.pkl')
    vectorizer = joblib.load('./vectorizer.pkl')
    v1=vectorizer.transform([wordToVec("东方溪谷经典干红葡萄酒西拉赤霞珠")]).toarray();
    print(classifier.predict(v1))
    v1=vectorizer.transform([wordToVec("嘉宝Gerber婴幼儿米粉米糊 宝宝辅食227g/罐 美国进口 DHA益生菌燕麦米粉227g")]).toarray();
    print(classifier.predict(v1))


C:\Users\liaomin\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liaomin\AppData\Local\Temp\jieba.cache
Loading model cost 0.918 seconds.
Prefix dict has been built successfully.


['2020000']
['1019903']
